# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: OPTIONAL to execute C++ code or Rust code</h2>
            <span style="color:#f71;">As an alternative, you can run it on the website given yesterday</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use high end models GPT 5, Claude 4.5 Sonnet, Gemini 2.5 Pro, Grok 4, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please pick lower cost models like gpt-5-nano.
            </span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import subprocess
from IPython.display import Markdown, display


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:6]}")
else:
    print("OpenRouter API Key not set (and this is optional)")



OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
Grok API Key not set (and this is optional)
Groq API Key not set (and this is optional)
OpenRouter API Key not set (and this is optional)


In [3]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"
groq_url = "https://api.groq.com/openai/v1"
openrouter_url = "https://openrouter.ai/api/v1"

import requests
requests.get("http://localhost:11434").content
OLLAMA_BASE_URL = "http://localhost:11434/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
openrouter = OpenAI(api_key=openrouter_api_key, base_url=openrouter_url)



In [4]:
models = ["gpt-5-nano", "claude-sonnet-4-5-20250929", "grok-4", "gemini-2.5-flash-lite", "qwen2.5", "deepseek", "llama3.2", "qwen/qwen3-coder-30b-a3b-instruct", "openai/gpt-oss-120b", ]

clients = {"gpt-5-nano": openai, "claude-sonnet-4-5-20250929": anthropic, "grok-4": grok, "gemini-2.5-flash-lite": gemini, "openai/gpt-oss-120b": groq, "qwen2.5": ollama, "deepseek": ollama, "llama3.2": ollama, "qwen/qwen3-coder-30b-a3b-instruct": openrouter}

# Want to keep costs ultra-low? Replace this with models of your choice, using the examples from yesterday

In [5]:
from system_info import retrieve_system_info, rust_toolchain_info

system_info = retrieve_system_info()
rust_info = rust_toolchain_info()
rust_info

{'installed': True,
 'rustc': {'path': 'C:\\Users\\Lenovo\\.cargo\\bin\\rustc.EXE',
  'version': 'rustc 1.93.0 (254b59607 2026-01-19)',
  'host_triple': 'x86_64-pc-windows-msvc',
  'release': '1.93.0',
  'commit_hash': '254b59607d4417e9dffbc307138ae5c86280fe4c'},
 'cargo': {'path': 'C:\\Users\\Lenovo\\.cargo\\bin\\cargo.EXE',
  'version': 'cargo 1.93.0 (083ac5135 2025-12-15)'},
 'rustup': {'path': 'C:\\Users\\Lenovo\\.cargo\\bin\\rustup.EXE',
  'version': 'rustup 1.28.2 (e4f3ad6f8 2025-04-28)',
  'active_toolchain': 'stable-x86_64-pc-windows-msvc (default)',
  'default_toolchain': '',
  'toolchains': ['stable-x86_64-pc-windows-msvc (active, default)'],
  'targets_installed': ['x86_64-pc-windows-msvc']},
 'rust_analyzer': {'path': 'C:\\Users\\Lenovo\\.cargo\\bin\\rust-analyzer.EXE'},
 'env': {'CARGO_HOME': 'C:\\Users\\Lenovo\\.cargo',
  'RUSTUP_HOME': 'C:\\Users\\Lenovo\\.rustup',
  'RUSTFLAGS': '',
  'CARGO_BUILD_TARGET': ''},
 'execution_examples': ['"C:\\Users\\Lenovo\\.cargo\\bin\\c

In [6]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Rust compiler to compile a single rust file called main.rs and then execute it in the simplest way possible.
Please reply with whether I need to install a Rust toolchain to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Rust code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.
Have the maximum possible runtime performance in mind; compile time can be slow. Fastest possible runtime performance for this platform is key.
Reply with the commands in markdown.

System information:
{system_info}

Rust toolchain information:
{rust_info}
"""

response = openai.chat.completions.create(model=models[0], messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

Short answer:
- You do not need to install anything new. Rust toolchain (rustc, cargo, rustup) is already installed and in use on your system.

What you currently have (as you shared):
- rustc.EXE at C:\Users\Lenovo\.cargo\bin\rustc.EXE
- cargo.EXE at C:\Users\Lenovo\.cargo\bin\cargo.EXE
- Active toolchain: stable-x86_64-pc-windows-msvc (default)

Given that, here is the simplest, fastest way to compile a single Rust file (main.rs) and run it, with the highest runtime performance in mind.

Step-by-step (manual commands you can run in PowerShell or CMD)
- Compile command (aimed at maximum runtime performance; uses native CPU optimizations, single codegen unit, and panic=abort):
- Run command (execute the resulting binary)

Python equivalents (exact commands to put into your script)

- Compile command:
```python
compile_command = [
    "C:\\Users\\Lenovo\\.cargo\\bin\\rustc.EXE",
    "main.rs",
    "-O",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "lto=fat",
    "-C", "panic=abort",
    "-o", "main.exe"
]
```

- Run command:
```python
run_command = ["main.exe"]
```

Notes and tips
- Run the compile command in the directory that contains main.rs. The produced executable will be main.exe in the same directory.
- On Windows, you execute the binary with either main.exe (CMD) or .\main.exe (PowerShell). In Python, using ["main.exe"] assumes the current working directory is the one containing main.exe.
- If you prefer the simplest setup (and you’re okay with cargo handling), you can put main.rs in a Cargo project and build with cargo build --release, then run the binary at target\release\YourProjectName.exe. This is often convenient for larger projects, but for a single file, rustc as shown above is the simplest fastest route to maximum performance.
- If you later add dependencies, consider switching to a Cargo-based workflow and use cargo build --release to ensure consistent optimizations.

## For C++, overwrite this with the commands from yesterday, or for Rust, use the new commands

Or just use the website like yesterday:

 https://www.programiz.com/cpp-programming/online-compiler/

In [8]:
compile_command = [
    "C:\\Users\\Lenovo\\.cargo\\bin\\rustc.EXE",
    "main.rs",
    "-O",
    "-C", "codegen-units=1",
    "-C", "target-cpu=native",
    "-C", "lto=fat",
    "-C", "panic=abort",
    "-o", "main.exe"
]
run_command = ["main.exe"]


## And now, on with the main task

In [9]:
language = "Rust" # or "C++"
extension = "rs" if language == "Rust" else "cpp"

system_prompt = f"""
Your task is to convert Python code into high performance {language} code.
Respond only with {language} code. Do not provide any explanation other than occasional comments.
The {language} response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to {language} with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.{language} and then compiled and executed; the compilation command is:
{compile_command}
Respond only with {language} code.
Python code to port:

```python
{python}
```
"""

In [10]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [11]:
def write_output(code):
    with open(f"main.{extension}", "w") as f:
        f.write(code)

In [12]:
def port(model, python):
    client = clients[model]
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```rust','').replace('```','')
    return reply

In [13]:
def run_python(code):
    globals_dict = {"__builtins__": __builtins__}

    buffer = io.StringIO()
    old_stdout = sys.stdout
    sys.stdout = buffer

    try:
        exec(code, globals_dict)
        output = buffer.getvalue()
    except Exception as e:
        output = f"Error: {e}"
    finally:
        sys.stdout = old_stdout

    return output

In [14]:
# Use the commands from GPT 5

def compile_and_run(code):
    write_output(code)
    try:
        subprocess.run(compile_command, check=True, text=True, capture_output=True)
        run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [15]:
python_hard = """# Be careful to support large numbers

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [16]:
from styles import CSS

with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:
    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python = gr.Code(
                label="Python (original)",
                value=python_hard,
                language="python",
                lines=26
            )
        with gr.Column(scale=6):
            cpp = gr.Code(
                label=f"{language} (generated)",
                value="",
                language="cpp",
                lines=26
            )

    with gr.Row(elem_classes=["controls"]):
        python_run = gr.Button("Run Python", elem_classes=["run-btn", "py"])
        model = gr.Dropdown(models, value=models[0], show_label=False)
        convert = gr.Button(f"Port to {language}", elem_classes=["convert-btn"])
        cpp_run = gr.Button(f"Run {language}", elem_classes=["run-btn", "cpp"])

    with gr.Row(equal_height=True):
        with gr.Column(scale=6):
            python_out = gr.TextArea(label="Python result", lines=8, elem_classes=["py-out"])
        with gr.Column(scale=6):
            cpp_out = gr.TextArea(label=f"{language} result", lines=8, elem_classes=["cpp-out"])

    convert.click(fn=port, inputs=[model, python], outputs=[cpp])
    python_run.click(fn=run_python, inputs=[python], outputs=[python_out])
    cpp_run.click(fn=compile_and_run, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19236\3949588770.py:3: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=CSS, theme=gr.themes.Monochrome(), title=f"Port from Python to {language}") as ui:


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Rust code by gpt
```rust
use std::time::Instant;

const A: u32 = 1664525;
const C: u32 = 1013904223;

fn max_subarray_sum(n: usize, seed: u32, min_val: i32, max_val: i32) -> i64 {
    let range = (max_val - min_val + 1) as u64; // e.g., 21
    let mut value: u32 = seed;
    let mut best: i64 = i64::MIN;
    let mut current: i64 = 0;

    for _ in 0..n {
        value = value.wrapping_mul(A).wrapping_add(C);
        let offset = (value as u64) % range;
        let num = (offset as i64) + (min_val as i64);

        current += num;
        if current > best {
            best = current;
        }
        if current < 0 {
            current = 0;
        }
    }

    best
}

fn total_max_subarray_sum(n: usize, initial_seed: u32, min_val: i32, max_val: i32) -> i64 {
    let mut value: u32 = initial_seed;
    let mut total_sum: i64 = 0;

    for _ in 0..20 {
        value = value.wrapping_mul(A).wrapping_add(C);
        total_sum += max_subarray_sum(n, value, min_val, max_val);
    }

    total_sum
}

fn main() {
    // Parameters matching the Python snippet
    let n: usize = 10_000;
    let initial_seed: u32 = 42;
    let min_val: i32 = -10;
    let max_val: i32 = 10;

    let start = Instant::now();
    let result = total_max_subarray_sum(n, initial_seed, min_val, max_val);
    let duration = start.elapsed();

    println!("Total Maximum Subarray Sum (20 runs): {}", result);
    println!("Execution Time: {:.6} seconds", duration.as_secs_f64());
}
```


## **Results**

## Rust
Total Maximum Subarray Sum (20 runs): 10980

Execution Time: 0.002737 seconds


## Python Outputs and time
Total Maximum Subarray Sum (20 runs): 10980

Execution Time: 321.343096 seconds

## C++ outputs and time
Total Maximum Subarray Sum (20 runs): 10980

Execution Time: 0.000912 seconds


## C++
```cpp
#include <iostream>
#include <chrono>
#include <cstdint>
#include <limits>

const uint32_t A = 1664525;
const uint32_t C = 1013904223;

int64_t max_subarray_sum(size_t n, uint32_t seed, int32_t min_val, int32_t max_val) {
    uint64_t range = static_cast<uint64_t>(max_val - min_val + 1);
    uint32_t value = seed;
    int64_t best = std::numeric_limits<int64_t>::min();
    int64_t current = 0;
    
    for (size_t i = 0; i < n; ++i) {
        value = value * A + C;
        uint64_t offset = static_cast<uint64_t>(value) % range;
        int64_t num = static_cast<int64_t>(offset) + static_cast<int64_t>(min_val);
        
        current += num;
        if (current > best) {
            best = current;
        }
        if (current < 0) {
            current = 0;
        }
    }
    
    return best;
}

int64_t total_max_subarray_sum(size_t n, uint32_t initial_seed, int32_t min_val, int32_t max_val) {
    uint32_t value = initial_seed;
    int64_t total_sum = 0;
    
    for (int i = 0; i < 20; ++i) {
        value = value * A + C;
        total_sum += max_subarray_sum(n, value, min_val, max_val);
    }
    
    return total_sum;
}

int main() {
    // Parameters matching the Rust code
    size_t n = 10000;
    uint32_t initial_seed = 42;
    int32_t min_val = -10;
    int32_t max_val = 10;
    
    auto start = std::chrono::high_resolution_clock::now();
    int64_t result = total_max_subarray_sum(n, initial_seed, min_val, max_val);
    auto end = std::chrono::high_resolution_clock::now();
    
    std::chrono::duration<double> duration = end - start;
    
    std::cout << "Total Maximum Subarray Sum (20 runs): " << result << std::endl;
    std::cout << "Execution Time: " << std::fixed << duration.count() << " seconds" << std::endl;
    
    return 0;
}
```